In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import ccf

import sys
sys.path.append("..")
from utils import *

In [9]:
data_URL = 'https://www.ssb.no/statbank/sq/10085910'

salmon_data = import_salmon_data(data_URL, rename_cols=True)

# Supply commodity data from yahoo finance
commodity_data = yf.download(['ZW=F', 'ZS=F'],
    start=salmon_data.index[0], 
    end=salmon_data.index[-1], 
    interval='1wk', 
    progress=False
)['Adj Close'].rename(columns={'ZW=F': 'Wheat', 'ZS=F': 'Soybean'})

data = pd.concat([salmon_data, commodity_data.reindex(salmon_data.index)], axis=1).dropna()

display(data)

display(data.corr())
data.to_csv('supply_data.csv')

,Price,Volume,Soybean,Wheat
Date,,,,
2000-09-11,29.51,5597,493.75,254.50
2000-09-18,29.33,5854,491.00,247.50
2000-09-25,29.18,6081,499.00,258.00
2000-10-02,28.44,5978,486.00,266.50
2000-10-09,27.76,5637,473.25,272.75
...,...,...,...,...
2023-07-24,86.14,19604,1486.75,704.25
2023-07-31,83.28,20469,1444.00,633.00
2023-08-07,80.26,21060,1401.25,626.75


,Price,Volume,Soybean,Wheat
Price,1.000000,0.601848,0.469542,0.349282
Volume,0.601848,1.000000,0.601081,0.499374
Soybean,0.469542,0.601081,1.000000,0.873001
Wheat,0.349282,0.499374,0.873001,1.000000


# Supply

### Fish Feed

- Soybean
- Wheat
- Fishmeal

In [10]:

def STL(data: pd.DataFrame, period: int=50):
    seasonal = seasonal_decompose(
        data, model="multiplicative", period=period
    )
    return seasonal

display(salmon_data.head())

models = {
    'Salmon': STL(data['Price'], period=50),
    'Wheat': STL(data['Wheat'], period=50), # Wheat
    'Soybean': STL(data['Soybean'], period=50), # Soybean
}

# for i, model in models.items():


cross_corr_ZW = ccf(models['Salmon'].seasonal, models['ZW=F'].seasonal)
cross_corr_ZS = ccf(models['Salmon'].seasonal, models['ZS=F'].seasonal)
cross_corr_HE = ccf(models['Salmon'].seasonal, models['HE=F'].seasonal)

fig, axs = plt.subplots(3,figsize=(10, 10))

axs[0].bar(range(52), cross_corr_ZW[:52])
axs[1].bar(range(52), cross_corr_ZS[:52])
axs[2].bar(range(52), cross_corr_HE[:52])

,Price,Volume
Date,,
2000-01-17,31.03,4043
2000-01-24,30.95,3730
2000-01-31,31.30,3831
2000-02-07,32.53,4415
2000-02-14,32.46,4617


KeyError: 'ZW=F'

## Demand

### substitute goods

- Poultry
- Pork
- Beef

### currency

- EUR/NOK